In [ ]:
import pandas as pd #pandas to create small dataframes 
import numpy as np #Do aritmetic operations on arrays
import matplotlib # matplotlib: used to plot graphs
#matplotlib.use('nbagg') : matplotlib uses this protocall which makes plots more user intractive like zoom in and zoom out
matplotlib.use('nbagg')
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams#Size of plots 
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import lightgbm as lgb

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Reading the Data Files**

In [ ]:
train =pd.read_csv('/content/drive/MyDrive/PGDUOH/Project/ashrae-energy-prediction/train.csv')
test = pd.read_csv('/content/drive/MyDrive/PGDUOH/Project/ashrae-energy-prediction/test.csv')
weather_train = pd.read_csv('/content/drive/MyDrive/PGDUOH/Project/ashrae-energy-prediction/weather_train.csv')
weather_test = pd.read_csv('/content/drive/MyDrive/PGDUOH/Project/ashrae-energy-prediction/weather_test.csv')
building_metadata = pd.read_csv('/content/drive/MyDrive/PGDUOH/Project/ashrae-energy-prediction/building_metadata.csv')
train.name = 'train'
test.name = 'test'
weather_train.name = 'weather_train'
weather_test.name = 'weather_test'
building_metadata.name = 'building_metadata'

**Pre-Processing**

In [ ]:
# Converting timestamp columns from all dataframes to datatime datatype
train['timestamp'] = pd.to_datetime(train['timestamp'])
test['timestamp'] = pd.to_datetime(test['timestamp'])
weather_train['timestamp'] = pd.to_datetime(weather_train['timestamp'])
weather_test['timestamp'] = pd.to_datetime(weather_test['timestamp'])

In [ ]:
# Function reducing dataframe size to fit into memory
# source : https://www.kaggle.com/c/ashrae-energy-prediction/discussion
def reduce_memory_usage(dataframe, verbose=True): 
    starting_memory = dataframe.memory_usage().sum() / 1024**2
    numeric_types = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    for col in dataframe:
        data_type = dataframe[col].dtype
        if data_type in numeric_types:
            min_val = dataframe[col].min() #taking min and max value from each column
            max_val = dataframe[col].max()
            # assigning datatype as per the min and max value in column            
            if str(data_type)[:3] == 'int':
                if min_val > np.iinfo('int8').min and max_val < np.iinfo('int8').max:
                    dataframe[col] = dataframe[col].astype('int8')
                elif min_val > np.iinfo('int16').min and max_val < np.iinfo('int16').max:
                    dataframe[col] = dataframe[col].astype('int16')
                elif min_val > np.iinfo('int32').min and max_val < np.iinfo('int32').max:
                    dataframe[col] = dataframe[col].astype('int32')
                else:
                    dataframe[col] = dataframe[col].astype('int64') 
            else: 
                if min_val > np.finfo('float16').min and max_val < np.finfo('float16').max:
                    dataframe[col] = dataframe[col].astype('float16')
                elif min_val > np.finfo('float32').min and max_val < np.finfo('float32').max:
                    dataframe[col] = dataframe[col].astype('float32')
                else: 
                    dataframe[col] = dataframe[col].astype('float64') # useless line?
    end_memory = dataframe.memory_usage().sum() / 1024**2
    if verbose:
        #Checking how much memory reduced after applying this code        
        print('Memory usage decreased to {:.2f} mb ({:.2f}% decrease)'.format(end_memory, 100 * (starting_memory-end_memory) / starting_memory))

In [ ]:
reduce_memory_usage(train)
reduce_memory_usage(test)
reduce_memory_usage(weather_train)
reduce_memory_usage(weather_test)
reduce_memory_usage(building_metadata)

Memory usage decreased to 289.19 mb (53.12% decrease)
Memory usage decreased to 596.49 mb (53.12% decrease)
Memory usage decreased to 3.07 mb (68.05% decrease)
Memory usage decreased to 6.08 mb (68.06% decrease)
Memory usage decreased to 0.03 mb (60.31% decrease)


In [ ]:
test.head()

,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01
1,1,1,0,2017-01-01
2,2,2,0,2017-01-01
3,3,3,0,2017-01-01
4,4,4,0,2017-01-01


In [ ]:
# Merging all training data frames using left join
train_merged = train.merge(building_metadata, on='building_id', how='left')
train_merged = train_merged.merge(weather_train, on=['site_id','timestamp'], how='left')

In [ ]:
# Merging all test data frames using left join
test_merged = test.merge(building_metadata, on='building_id', how='left')
test_merged = test_merged.merge(weather_test, on=['site_id','timestamp'], how='left')

In [ ]:
train_merged.describe()

,building_id,meter,meter_reading,site_id,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
count,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,8088455.0,3506933.0,2.011944e+07,11390735.0,2.011596e+07,16467077.0,18984431.0,18767052.0,2.007242e+07
mean,7.992780e+02,6.624412e-01,1.988706e+03,7.992232e+00,1.077830e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,4.269133e+02,9.309921e-01,1.532159e+05,5.099060e+00,1.171424e+05,NaN,0.0,0.000000e+00,0.0,0.000000e+00,NaN,NaN,NaN,0.000000e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.830000e+02,1900.0,1.0,-2.890625e+01,0.0,-3.500000e+01,-1.0,968.0,0.0,0.000000e+00
25%,3.930000e+02,0.000000e+00,1.830000e+01,3.000000e+00,3.252700e+04,1951.0,1.0,8.601562e+00,0.0,0.000000e+00,0.0,1011.5,70.0,2.099609e+00
50%,8.950000e+02,0.000000e+00,7.877500e+01,9.000000e+00,7.270900e+04,1969.0,3.0,1.670312e+01,0.0,8.898438e+00,0.0,1016.0,180.0,3.099609e+00
75%,1.179000e+03,1.000000e+00,2.679840e+02,1.300000e+01,1.391130e+05,1993.0,6.0,2.409375e+01,4.0,1.609375e+01,0.0,1020.5,280.0,4.601562e+00
max,1.448000e+03,3.000000e+00,2.190470e+07,1.500000e+01,8.750000e+05,2017.0,26.0,4.718750e+01,9.0,2.609375e+01,343.0,1046.0,360.0,1.900000e+01


In [ ]:
test_merged.describe()

,row_id,building_id,meter,site_id,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
count,4.169760e+07,4.169760e+07,4.169760e+07,4.169760e+07,4.169760e+07,17099520.0,7253280.0,4.147570e+07,22155420.0,4.143680e+07,33896037.0,39180774.0,38718937.0,4.139551e+07
mean,2.084880e+07,8.075824e+02,6.642857e-01,8.086134e+00,1.069469e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,1.203706e+07,4.297680e+02,9.278067e-01,5.134712e+00,1.160888e+05,NaN,0.0,0.000000e+00,0.0,0.000000e+00,NaN,NaN,NaN,0.000000e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.830000e+02,1900.0,1.0,-2.809375e+01,0.0,-3.159375e+01,-1.0,972.0,0.0,0.000000e+00
25%,1.042440e+07,4.047500e+02,0.000000e+00,3.000000e+00,3.224350e+04,1951.0,1.0,7.800781e+00,0.0,0.000000e+00,0.0,1011.5,80.0,2.099609e+00
50%,2.084880e+07,9.000000e+02,0.000000e+00,9.000000e+00,7.226250e+04,1969.0,3.0,1.670312e+01,2.0,8.898438e+00,0.0,1016.0,180.0,3.099609e+00
75%,3.127320e+07,1.194250e+03,1.000000e+00,1.300000e+01,1.383875e+05,1993.0,6.0,2.390625e+01,4.0,1.670312e+01,0.0,1020.5,280.0,4.601562e+00
max,4.169760e+07,1.448000e+03,3.000000e+00,1.500000e+01,8.750000e+05,2017.0,26.0,4.831250e+01,9.0,2.670312e+01,597.0,1050.0,360.0,2.420312e+01


**Outlier removal**

In [ ]:
# Outlier removal for building id 1099
train_merged[train_merged['building_id']==1099]['meter_reading'].describe()
index= list(train_merged[train_merged['building_id']==1099].index)
train_merged.drop(index, axis=0, inplace=True)

In [ ]:
# Outlier removal for building id 778
train_merged[train_merged['building_id']==778]['meter_reading'].describe()
index= list(train_merged[train_merged['building_id']==778].index)
train_merged.drop(index, axis=0, inplace=True)

In [ ]:
# removing the data points with zero readings
train_merged.shape
zero_readings= list(train_merged[train_merged['meter_reading']==0.0].index)
print(len(zero_readings))
train_merged.drop(zero_readings, axis=0, inplace=True)
train_merged.shape

1866732


(18314942, 16)

**New Features from timestamp column**

In [ ]:
#Creating fetures from timestamp column on train data
train_merged['month'] = train_merged.timestamp.dt.month
train_merged['weekday'] = train_merged.timestamp.dt.weekday
train_merged['hour'] = train_merged.timestamp.dt.hour

In [ ]:
#Creating fetures from timestamp column on test data
test_merged['month'] = test_merged.timestamp.dt.month
test_merged['weekday'] = test_merged.timestamp.dt.weekday
test_merged['hour'] = test_merged.timestamp.dt.hour

**Missing Value Imputation**

In [ ]:
def missing_value_imp(df):
  # year_built: Filling na values with mean 
  year_built=df.groupby(['site_id'])['year_built'].transform('mean')
  df['year_built'].fillna(year_built, inplace=True)

  # floor_count: Filling na values with mean 
  floor_count=df.groupby(['site_id'])['floor_count'].transform('mean')
  # floor_count
  df['floor_count'].fillna(floor_count, inplace=True)

  # there are some sites for which still floor_count value in na, so imputing them with one (min floors a building can have)
  floor_count=1 
  # floor_count
  df['floor_count'].fillna(floor_count, inplace=True)

  # year_built: Filling na values with meadian
  year_built=df['year_built'].median()
  # year_built
  df['year_built'].fillna(year_built, inplace=True)

  # air_temperature: Filling na values with meadian
  air_temperature=df.groupby(['site_id', 'weekday', 'month'])['air_temperature'].transform('median')
  df['air_temperature'].fillna(air_temperature, inplace=True)

  # cloud_coverage: Filling na values with meadian
  cloud_coverage=df.groupby(['site_id', 'weekday', 'month'])['cloud_coverage'].transform('median')
  df['cloud_coverage'].fillna(cloud_coverage, inplace=True)

  # dew_temperature: Filling na values with meadian
  dew_temperature=df.groupby(['site_id', 'weekday', 'month'])['dew_temperature'].transform('median')
  df['dew_temperature'].fillna(dew_temperature, inplace=True)

  # precip_depth_1_hr: Filling na values with meadian
  precip_depth_1_hr=df.groupby(['site_id', 'weekday', 'month'])['precip_depth_1_hr'].transform('median')
  df['precip_depth_1_hr'].fillna(precip_depth_1_hr, inplace=True)

  # there are some sites for which cloud_coverage value is still na, so imputing them with overall median value
  cloud_coverage=df['cloud_coverage'].median()
  # cloud_coverage
  df['cloud_coverage'].fillna(cloud_coverage, inplace=True)

  # there are some sites for which precip_depth_1_hr value is still na, so imputing them with overall median value
  precip_depth_1_hr=df['precip_depth_1_hr'].median()
  # precip_depth_1_hr
  df['precip_depth_1_hr'].fillna(precip_depth_1_hr, inplace=True)

  # sea_level_pressure: Filling na values with mean
  sea_level=df.groupby(['site_id', 'weekday', 'month'])['sea_level_pressure'].transform('mean')
  df['sea_level_pressure'].fillna(sea_level, inplace=True)

  # wind_direction: Filling na values with mean
  wind_direction=df.groupby(['site_id', 'weekday', 'month'])['wind_direction'].transform('mean')
  df['wind_direction'].fillna(wind_direction, inplace=True)

  # wind_speed: Filling na values with mean
  wind_speed=df.groupby(['site_id', 'weekday', 'month'])['wind_speed'].transform('mean')
  df['wind_speed'].fillna(wind_speed, inplace=True)

  # there are some sites for which sea_level_pressure value is still na, so imputing them with overall median value
  sea_level_pressure=df['sea_level_pressure'].median()
  df['sea_level_pressure'].fillna(sea_level_pressure, inplace=True)


  return df


In [ ]:
train_merged= missing_value_imp(train_merged)

In [ ]:
test_merged= missing_value_imp(test_merged)

In [ ]:
train_merged.isna().sum()

building_id           0
meter                 0
timestamp             0
meter_reading         0
site_id               0
primary_use           0
square_feet           0
year_built            0
floor_count           0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
month                 0
weekday               0
hour                  0
dtype: int64

In [ ]:
test_merged.isna().sum()

row_id                0
building_id           0
meter                 0
timestamp             0
site_id               0
primary_use           0
square_feet           0
year_built            0
floor_count           0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
month                 0
weekday               0
hour                  0
dtype: int64

# Feature Engineering

In [ ]:
# Creating if a date is holiday feature: train
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

df = pd.DataFrame()
df['Date'] = train_merged.timestamp.dt.date
# print(df['Date'].min())
# print(df['Date'].max())

cal = calendar()
holidays = cal.holidays(start=df['Date'].min(), end=df['Date'].max())

# df['Holiday'] = df['Date'].isin(holidays)
# print (df)
train_merged['is_holiday'] = df['Date'].isin(holidays)
train_merged.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/algorithms.py:524: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  return f(comps, values)


,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,month,weekday,hour,is_holiday
45,46,0,2016-01-01,53.239700,0,Retail,9045,2016.0,1.0,25.000000,6.0,20.000000,0.0,1019.5,0.0,0.000000,1,4,0,False
72,74,0,2016-01-01,43.001301,0,Parking,387638,1997.0,1.0,25.000000,6.0,20.000000,0.0,1019.5,0.0,0.000000,1,4,0,False
91,93,0,2016-01-01,52.420601,0,Office,33370,1982.0,1.0,25.000000,6.0,20.000000,0.0,1019.5,0.0,0.000000,1,4,0,False
103,105,0,2016-01-01,23.303600,1,Education,50623,1961.0,5.0,3.800781,0.0,2.400391,0.0,1021.0,240.0,3.099609,1,4,0,False
104,106,0,2016-01-01,0.374600,1,Education,5374,1961.0,4.0,3.800781,0.0,2.400391,0.0,1021.0,240.0,3.099609,1,4,0,False


In [ ]:
# Creating if a date is holiday feature: test
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

df = pd.DataFrame()
df['Date'] = test_merged.timestamp.dt.date

cal = calendar()
holidays = cal.holidays(start=df['Date'].min(), end=df['Date'].max())

test_merged['is_holiday'] = df['Date'].isin(holidays)
test_merged.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/algorithms.py:524: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  return f(comps, values)


,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,month,weekday,hour,is_holiday
0,0,0,0,2017-01-01,0,Education,7432,2008.0,1.0,17.796875,4.0,11.703125,0.0,1021.5,100.0,3.599609,1,6,0,False
1,1,1,0,2017-01-01,0,Education,2720,2004.0,1.0,17.796875,4.0,11.703125,0.0,1021.5,100.0,3.599609,1,6,0,False
2,2,2,0,2017-01-01,0,Education,5376,1991.0,1.0,17.796875,4.0,11.703125,0.0,1021.5,100.0,3.599609,1,6,0,False
3,3,3,0,2017-01-01,0,Education,23685,2002.0,1.0,17.796875,4.0,11.703125,0.0,1021.5,100.0,3.599609,1,6,0,False
4,4,4,0,2017-01-01,0,Education,116607,1975.0,1.0,17.796875,4.0,11.703125,0.0,1021.5,100.0,3.599609,1,6,0,False


In [ ]:
# Adding season feature based on US seasons
# train
# Summer: June - August Fall: September - November Winter: December - February Spring: March - May
train_merged['season']= train_merged['month'].apply(lambda x: 'Spring' if x==3 or x==4 or x==5 else 'Summer' if 
                                                x==6 or x==7 or x==8 
                                                else 'Autumn' if x==9 or x==10 or 
                                                x==11 else 'Winter')

In [ ]:
# Adding season feature based on US seasons
# test
# Summer: June - August Fall: September - November Winter: December - February Spring: March - May
test_merged['season']= test_merged['month'].apply(lambda x: 'Spring' if x==3 or x==4 or x==5 else 'Summer' if 
                                                x==6 or x==7 or x==8 
                                                else 'Autumn' if x==9 or x==10 or 
                                                x==11 else 'Winter')

In [ ]:
# Adding if its day time hours or not feature
# train
train_merged['IsDay']= train_merged['hour'].apply(lambda x: 1 if x >=6 and x <=18 else 0)

In [ ]:
# Adding if its day time hours or not feature
# test
test_merged['IsDay']= test_merged['hour'].apply(lambda x: 1 if x >=6 and x <=18 else 0)

In [ ]:
#Adding relative humidity
# train
# Source: https://medium.com/analytics-vidhya/ashrae-great-energy-predictor-iii-a-machine-learning-case-study-a01a67eb048d
train_merged['relative_humidity']= 100*((np.exp((17.67*train_merged['dew_temperature'])/
                                            (243.5+train_merged['dew_temperature'])))/(np.exp((17.67*train_merged['air_temperature'])/
                                                                                          (243.5+train_merged['air_temperature']))))

In [ ]:
#Adding relative humidity
# test
# Source: https://medium.com/analytics-vidhya/ashrae-great-energy-predictor-iii-a-machine-learning-case-study-a01a67eb048d
test_merged['relative_humidity']= 100*((np.exp((17.67*test_merged['dew_temperature'])/
                                            (243.5+test_merged['dew_temperature'])))/(np.exp((17.67*test_merged['air_temperature'])/
                                                                                          (243.5+test_merged['air_temperature']))))

In [ ]:
train_merged.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,month,weekday,hour,is_holiday,season,IsDay,relative_humidity
45,46,0,2016-01-01,53.239700,0,Retail,9045,2016.0,1.0,25.000000,6.0,20.000000,0.0,1019.5,0.0,0.000000,1,4,0,False,Winter,0,73.8125
72,74,0,2016-01-01,43.001301,0,Parking,387638,1997.0,1.0,25.000000,6.0,20.000000,0.0,1019.5,0.0,0.000000,1,4,0,False,Winter,0,73.8125
91,93,0,2016-01-01,52.420601,0,Office,33370,1982.0,1.0,25.000000,6.0,20.000000,0.0,1019.5,0.0,0.000000,1,4,0,False,Winter,0,73.8125
103,105,0,2016-01-01,23.303600,1,Education,50623,1961.0,5.0,3.800781,0.0,2.400391,0.0,1021.0,240.0,3.099609,1,4,0,False,Winter,0,90.5000
104,106,0,2016-01-01,0.374600,1,Education,5374,1961.0,4.0,3.800781,0.0,2.400391,0.0,1021.0,240.0,3.099609,1,4,0,False,Winter,0,90.5000


In [ ]:
test_merged.head()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,month,weekday,hour,is_holiday,season,IsDay,relative_humidity
0,0,0,0,2017-01-01,0,Education,7432,2008.0,1.0,17.796875,4.0,11.703125,0.0,1021.5,100.0,3.599609,1,6,0,False,Winter,0,67.5
1,1,1,0,2017-01-01,0,Education,2720,2004.0,1.0,17.796875,4.0,11.703125,0.0,1021.5,100.0,3.599609,1,6,0,False,Winter,0,67.5
2,2,2,0,2017-01-01,0,Education,5376,1991.0,1.0,17.796875,4.0,11.703125,0.0,1021.5,100.0,3.599609,1,6,0,False,Winter,0,67.5
3,3,3,0,2017-01-01,0,Education,23685,2002.0,1.0,17.796875,4.0,11.703125,0.0,1021.5,100.0,3.599609,1,6,0,False,Winter,0,67.5
4,4,4,0,2017-01-01,0,Education,116607,1975.0,1.0,17.796875,4.0,11.703125,0.0,1021.5,100.0,3.599609,1,6,0,False,Winter,0,67.5


In [ ]:
# train_merged.reset_index().to_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/train_merged_01.ftr')

In [ ]:
# test_merged.reset_index().to_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/test_merged_01.ftr')

In [ ]:
train_merged.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,month,weekday,hour,is_holiday,season,IsDay,relative_humidity
45,46,0,2016-01-01,53.239700,0,Retail,9045,2016.0,1.0,25.000000,6.0,20.000000,0.0,1019.5,0.0,0.000000,1,4,0,False,Winter,0,73.8125
72,74,0,2016-01-01,43.001301,0,Parking,387638,1997.0,1.0,25.000000,6.0,20.000000,0.0,1019.5,0.0,0.000000,1,4,0,False,Winter,0,73.8125
91,93,0,2016-01-01,52.420601,0,Office,33370,1982.0,1.0,25.000000,6.0,20.000000,0.0,1019.5,0.0,0.000000,1,4,0,False,Winter,0,73.8125
103,105,0,2016-01-01,23.303600,1,Education,50623,1961.0,5.0,3.800781,0.0,2.400391,0.0,1021.0,240.0,3.099609,1,4,0,False,Winter,0,90.5000
104,106,0,2016-01-01,0.374600,1,Education,5374,1961.0,4.0,3.800781,0.0,2.400391,0.0,1021.0,240.0,3.099609,1,4,0,False,Winter,0,90.5000


**Normalization**

In [ ]:
# Normalizing columns using log1P normalization
train_merged['floor_count'] = np.log1p(train_merged['floor_count'])
train_merged['square_feet'] = np.log1p(train_merged['square_feet'])
train_merged['meter_reading'] = np.log1p(train_merged['meter_reading'])

test_merged['floor_count'] = np.log1p(test_merged['floor_count'])
test_merged['square_feet'] = np.log1p(test_merged['square_feet'])

In [ ]:
# train_merged.reset_index().to_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/train_merged_02.ftr')
# test_merged.to_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/test_merged_02.ftr')

In [ ]:
print(train_merged['meter_reading'].min())
print(train_merged['meter_reading'].max())

9.999499889090657e-05
12.78448486328125


In [ ]:
# # Normalizing meter_reading column using 0-1 scaling normalization
# from sklearn.preprocessing import minmax_scale
# train_merged['meter_reading'] = minmax_scale(train_merged['meter_reading'], feature_range=(0, 1))  # 0-1 scaling

**Time based splitting of train data**

In [ ]:
train_merged=train_merged.sort_values(by='timestamp')

In [ ]:
X_train, X_cv= train_test_split(train_merged, test_size=0.20, shuffle=False)

**Label Encoding**

In [ ]:
# Label encoding primary use feature
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_merged.primary_use)
# LabelEncoder()
list(le.classes_)

X_train.primary_use = le.transform(X_train.primary_use)
X_cv.primary_use = le.transform(X_cv.primary_use)
test_merged.primary_use = le.transform(test_merged.primary_use)

In [ ]:
# Label encoding season feature
le.fit(train_merged.season)

X_train.season = le.transform(X_train.season)
X_cv.season = le.transform(X_cv.season)
test_merged.season = le.transform(test_merged.season)

In [ ]:
# Encoding is_holiday feature to binary form
X_train["is_holiday"] = X_train["is_holiday"].astype(int)
X_cv["is_holiday"] = X_cv["is_holiday"].astype(int)
test_merged["is_holiday"] = test_merged["is_holiday"].astype(int)

In [ ]:
# X_train.reset_index().to_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/X_train_01.ftr')
# X_cv.reset_index().to_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/X_cv_01.ftr')
# test_merged.reset_index().to_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/test_merged_03.ftr')

In [ ]:
Y_train=X_train['meter_reading']
Y_cv=X_cv['meter_reading']
X_train.drop(['meter_reading'], axis=1, inplace=True)
X_cv.drop(['meter_reading'], axis=1, inplace=True)

**Variance Inflation Factor (VIF_ for Multicolinearity)**

In [ ]:
#https://etav.github.io/python/vif_factor_python.html
#http://statisticshowto.com/variance-inflation-factor/

from statsmodels.stats.outliers_influence import variance_inflation_factor
s=X_cv._get_numeric_data() #calculating for cross validation data because for train data size collab was crashing
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(s.values, i) for i in range(s.shape[1])]
vif["features"] = s.columns

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
vif

,VIF Factor,features
0,138.855195,building_id
1,1.721370,meter
2,109.382172,site_id
3,2.012092,primary_use
4,101.965979,square_feet
5,7912.996309,year_built
6,7.572761,floor_count
7,94.926986,air_temperature
8,1.816901,cloud_coverage
9,50.464114,dew_temperature


**Dropping the fetures with higher VIF factor**

In [ ]:
X_train.drop(['building_id', 'air_temperature','timestamp','sea_level_pressure'],axis=1,inplace=True)
X_cv.drop(['building_id', 'air_temperature','timestamp','sea_level_pressure'],axis=1,inplace=True)

In [ ]:
test_merged.drop(['building_id', 'air_temperature','timestamp','sea_level_pressure'],axis=1,inplace=True)

In [ ]:
X_train.head()

,meter,site_id,primary_use,square_feet,year_built,floor_count,cloud_coverage,dew_temperature,precip_depth_1_hr,wind_direction,wind_speed,month,weekday,hour,is_holiday,season,IsDay,relative_humidity
45,0,0,11,9.110078,2016.0,0.693359,6.0,20.000000,0.0,0.0,0.000000,1,4,0,0,3,0,73.8125
1663,0,13,6,12.427046,1965.0,0.693359,8.0,-12.203125,0.0,250.0,4.101562,1,4,0,0,3,0,73.3750
1662,2,13,8,11.337619,1965.0,0.693359,8.0,-12.203125,0.0,250.0,4.101562,1,4,0,0,3,0,73.3750
1661,1,13,8,11.337619,1965.0,0.693359,8.0,-12.203125,0.0,250.0,4.101562,1,4,0,0,3,0,73.3750
1660,0,13,8,11.337619,1965.0,0.693359,8.0,-12.203125,0.0,250.0,4.101562,1,4,0,0,3,0,73.3750


In [ ]:
# Saving pre-processed data files
X_train.reset_index().to_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/X_train_F.ftr')
X_cv.reset_index().to_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/X_cv_F.ftr')
test_merged.reset_index().to_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/test_merged_F.ftr')

In [ ]:
#Converting pandas series to numpy
Y_train=Y_train.to_numpy()
Y_cv=Y_cv.to_numpy()

In [ ]:
# Saving pre-processed data files
np.save('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/Y_train.npy', Y_train)
np.save('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/Y_cv.npy',Y_cv)

In [ ]:
# X_train= pd.read_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/X_train_F.ftr')
# X_cv= pd.read_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/X_cv_F.ftr')

In [ ]:
# X_train= pd.DataFrame(data=X_train)

In [ ]:
# Y_train= np.load('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/Y_train.npy')
# Y_cv= np.load('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/Y_cv.npy')

In [ ]:
Y_train

array([3.9934132, 5.492349 , 6.482104 , ..., 5.697087 , 4.658142 ,
       2.0806909], dtype=float32)

**LGB GBDT**

In [ ]:
# Source: https://sefiks.com/2018/10/13/a-gentle-introduction-to-lightgbm-for-applied-machine-learning/
# https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.train.html
# num_boost_round=50
cv_RMSLE=[]
lgb_train = lgb.Dataset(X_train, Y_train
,feature_name =  ['meter','site_id','primary_use','square_feet','year_built','floor_count','cloud_coverage','dew_temperature','precip_depth_1_hr','wind_direction','wind_speed','month','weekday','hour','is_holiday','season','IsDay','relative_humidity']  
, categorical_feature = ['primary_use','is_holiday','season']
)

In [ ]:
params = {
'task': 'train'
, 'boosting_type': 'gbdt'
, 'objective': 'regression' 
, 'metric': 'rmsle'
, 'subsample': 0.8
,'feature_fraction':0.8
,'bagging_freq': 1
, 'min_data': 1
, 'verbose': -1
}
 
gbm = lgb.train(params, lgb_train, num_boost_round=50)

/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


In [ ]:
# Prediction on CV data
gbm_cv=gbm.predict(X_cv)
# # Normalizing meter_reading column using 0-1 scaling normalization
from sklearn.preprocessing import minmax_scale
Y_cv = minmax_scale(Y_cv, feature_range=(0, 1))  # 0-1 scaling
gbm_cv = minmax_scale(gbm_cv, feature_range=(0, 1))  # 0-1 scaling

cv_RMSLE.append(np.sqrt(mean_squared_log_error(Y_cv, gbm_cv)))
print(cv_RMSLE)

[0.08829917829913075]


In [ ]:
# Source: https://sefiks.com/2018/10/13/a-gentle-introduction-to-lightgbm-for-applied-machine-learning/
# num_boost_round=1000
cv_RMSLE=[]
lgb_train = lgb.Dataset(X_train, Y_train
,feature_name =  ['meter','site_id','primary_use','square_feet','year_built','floor_count','cloud_coverage','dew_temperature','precip_depth_1_hr','wind_direction','wind_speed','month','weekday','hour','is_holiday','season','IsDay','relative_humidity']  
, categorical_feature = ['primary_use','is_holiday','season']
)

In [ ]:
params = {
'task': 'train'
, 'boosting_type': 'gbdt'
, 'objective': 'regression' 
, 'metric': 'rmsle'
, 'subsample': 0.8
,'feature_fraction':0.8
,'bagging_freq': 1
, 'min_data': 1
, 'verbose': -1
}
 
gbm = lgb.train(params, lgb_train, num_boost_round=1000)

/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


In [ ]:
# Prediction on CV data
gbm_cv=gbm.predict(X_cv)
# # Normalizing meter_reading column using 0-1 scaling normalization
from sklearn.preprocessing import minmax_scale
Y_cv = minmax_scale(Y_cv, feature_range=(0, 1))  # 0-1 scaling
gbm_cv = minmax_scale(gbm_cv, feature_range=(0, 1))  # 0-1 scaling

cv_RMSLE.append(np.sqrt(mean_squared_log_error(Y_cv, gbm_cv)))
print(cv_RMSLE)

[0.05663832223626501]


In [ ]:
# Saving the trained model
import joblib
filename_reg='lgb_reg.sav'
joblib.dump(gbm,filename_reg)

['lgb_reg.sav']

**Catboost Regressor**

In [ ]:
# !pip install catboost

     |████████████████████████████████| 76.1 MB 76 kB/s 


In [ ]:
# estimators: 50
cv_rmsle=[]
import catboost as cb
cgbdt=cb.CatBoostRegressor(n_estimators=50)
cgbdt.fit(X_train,Y_train)
cgbdt_cv=cgbdt.predict(X_cv)
Y_cv = minmax_scale(Y_cv, feature_range=(0, 1))  # 0-1 scaling
cgbdt_cv = minmax_scale(cgbdt_cv, feature_range=(0, 1))  # 0-1 scaling
cv_rmsle.append(np.sqrt(mean_squared_log_error(Y_cv, cgbdt_cv)))
print("rmsle for estimators:", 50,'is',np.sqrt(mean_squared_log_error(Y_cv, cgbdt_cv)))

Learning rate set to 0.5
0:	learn: 1.4451731	total: 1.29s	remaining: 1m 3s
1:	learn: 1.3273492	total: 2.33s	remaining: 55.8s
2:	learn: 1.2585467	total: 3.3s	remaining: 51.7s
3:	learn: 1.2181369	total: 4.23s	remaining: 48.7s
4:	learn: 1.1810262	total: 5.21s	remaining: 46.9s
5:	learn: 1.1595430	total: 6.13s	remaining: 44.9s
6:	learn: 1.1436038	total: 7.16s	remaining: 44s
7:	learn: 1.1311812	total: 8.11s	remaining: 42.6s
8:	learn: 1.1069319	total: 9.05s	remaining: 41.2s
9:	learn: 1.0851645	total: 9.99s	remaining: 40s
10:	learn: 1.0712636	total: 11s	remaining: 38.8s
11:	learn: 1.0620120	total: 12s	remaining: 37.9s
12:	learn: 1.0547334	total: 12.9s	remaining: 36.7s
13:	learn: 1.0500234	total: 13.7s	remaining: 35.3s
14:	learn: 1.0301302	total: 14.7s	remaining: 34.4s
15:	learn: 1.0246802	total: 15.7s	remaining: 33.4s
16:	learn: 1.0177758	total: 16.7s	remaining: 32.3s
17:	learn: 1.0136439	total: 17.5s	remaining: 31.1s
18:	learn: 1.0091207	total: 18.4s	remaining: 30s
19:	learn: 1.0043296	total:

In [ ]:
# estimators: 1000
cv_rmsle=[]
import catboost as cb
cgbdt=cb.CatBoostRegressor(n_estimators=1000)
cgbdt.fit(X_train,Y_train)
cgbdt_cv=cgbdt.predict(X_cv)
Y_cv = minmax_scale(Y_cv, feature_range=(0, 1))  # 0-1 scaling
cgbdt_cv = minmax_scale(cgbdt_cv, feature_range=(0, 1))  # 0-1 scaling
cv_rmsle.append(np.sqrt(mean_squared_log_error(Y_cv, cgbdt_cv)))
print("rmsle for estimators:", 1000,'is',np.sqrt(mean_squared_log_error(Y_cv, cgbdt_cv)))

Learning rate set to 0.186376
0:	learn: 1.6252223	total: 1.17s	remaining: 19m 28s
1:	learn: 1.5286710	total: 2.18s	remaining: 18m 8s
2:	learn: 1.4561469	total: 3.16s	remaining: 17m 31s
3:	learn: 1.4043162	total: 4.14s	remaining: 17m 11s
4:	learn: 1.3648773	total: 5.15s	remaining: 17m 4s
5:	learn: 1.3304785	total: 6.14s	remaining: 16m 57s
6:	learn: 1.2888849	total: 7.2s	remaining: 17m 1s
7:	learn: 1.2692014	total: 8.18s	remaining: 16m 53s
8:	learn: 1.2425776	total: 9.18s	remaining: 16m 50s
9:	learn: 1.2293793	total: 10.1s	remaining: 16m 39s
10:	learn: 1.2136934	total: 11.1s	remaining: 16m 35s
11:	learn: 1.1962626	total: 12s	remaining: 16m 31s
12:	learn: 1.1842293	total: 13s	remaining: 16m 23s
13:	learn: 1.1738067	total: 14s	remaining: 16m 25s
14:	learn: 1.1664622	total: 15s	remaining: 16m 22s
15:	learn: 1.1546499	total: 15.9s	remaining: 16m 16s
16:	learn: 1.1390573	total: 16.9s	remaining: 16m 15s
17:	learn: 1.1264481	total: 17.9s	remaining: 16m 15s
18:	learn: 1.1193764	total: 18.9s	rema

In [ ]:
# Reading pre-processed data from ftr files
X_train_f= pd.read_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/X_train_F.ftr')
X_cv_f= pd.read_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/X_cv_F.ftr')

In [ ]:
Y_train_f= np.load('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/Y_train.npy')
Y_cv_f= np.load('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/Y_cv.npy')

In [ ]:
# Y_cv_f
X_train_f.drop(['index'], axis=1, inplace=True)
X_cv_f.drop(['index'], axis=1, inplace=True)
X_train_f.head()

,meter,site_id,primary_use,square_feet,year_built,floor_count,cloud_coverage,dew_temperature,precip_depth_1_hr,wind_direction,wind_speed,month,weekday,hour,is_holiday,season,IsDay,relative_humidity
0,0,0,11,9.110078,2016.0,0.693359,6.0,20.000000,0.0,0.0,0.000000,1,4,0,0,3,0,73.8125
1,0,13,6,12.427046,1965.0,0.693359,8.0,-12.203125,0.0,250.0,4.101562,1,4,0,0,3,0,73.3750
2,2,13,8,11.337619,1965.0,0.693359,8.0,-12.203125,0.0,250.0,4.101562,1,4,0,0,3,0,73.3750
3,1,13,8,11.337619,1965.0,0.693359,8.0,-12.203125,0.0,250.0,4.101562,1,4,0,0,3,0,73.3750
4,0,13,8,11.337619,1965.0,0.693359,8.0,-12.203125,0.0,250.0,4.101562,1,4,0,0,3,0,73.3750


In [ ]:
# estimators: 1200
cv_rmsle=[]
import catboost as cb
cgbdt_f=cb.CatBoostRegressor(n_estimators=1200)
cgbdt_f.fit(X_train_f,Y_train_f)
cgbdt_cv_f=cgbdt_f.predict(X_cv_f)
from sklearn.preprocessing import minmax_scale
Y_cv_f = minmax_scale(Y_cv_f, feature_range=(0, 1))  # 0-1 scaling
cgbdt_cv_f = minmax_scale(cgbdt_cv_f, feature_range=(0, 1))  # 0-1 scaling
cv_rmsle.append(np.sqrt(mean_squared_log_error(Y_cv_f, cgbdt_cv_f)))
print("rmsle for estimators:", 1200,'is',np.sqrt(mean_squared_log_error(Y_cv_f, cgbdt_cv_f)))

Learning rate set to 0.1607
0:	learn: 1.6426492	total: 1.56s	remaining: 31m 17s
1:	learn: 1.5544244	total: 2.84s	remaining: 28m 20s
2:	learn: 1.4876681	total: 4.12s	remaining: 27m 22s
3:	learn: 1.4352206	total: 5.36s	remaining: 26m 42s
4:	learn: 1.3804040	total: 6.67s	remaining: 26m 34s
5:	learn: 1.3475286	total: 7.84s	remaining: 26m 1s
6:	learn: 1.3152423	total: 9.06s	remaining: 25m 44s
7:	learn: 1.2888834	total: 10.2s	remaining: 25m 18s
8:	learn: 1.2686479	total: 11.4s	remaining: 25m 6s
9:	learn: 1.2551391	total: 12.6s	remaining: 24m 56s
10:	learn: 1.2331795	total: 13.8s	remaining: 24m 51s
11:	learn: 1.2125616	total: 15.2s	remaining: 25m 2s
12:	learn: 1.2020109	total: 16.3s	remaining: 24m 50s
13:	learn: 1.1931388	total: 17.5s	remaining: 24m 44s
14:	learn: 1.1777342	total: 18.7s	remaining: 24m 36s
15:	learn: 1.1699144	total: 19.9s	remaining: 24m 33s
16:	learn: 1.1604502	total: 21s	remaining: 24m 21s
17:	learn: 1.1547968	total: 22.1s	remaining: 24m 13s
18:	learn: 1.1457295	total: 23.3s

In [ ]:
import joblib
filename_reg='/content/drive/MyDrive/PGDUOH/Project/catboost_reg_1200.sav'
joblib.dump(cgbdt_f,filename_reg)

['/content/drive/MyDrive/PGDUOH/Project/catboost_reg_1200.sav']

**LGB GBDT with 500 rounds**

In [ ]:
# Source: https://sefiks.com/2018/10/13/a-gentle-introduction-to-lightgbm-for-applied-machine-learning/
cv_RMSLE=[]
lgb_train_f = lgb.Dataset(X_train_f, Y_train_f
,feature_name =  ['meter','site_id','primary_use','square_feet','year_built','floor_count','cloud_coverage','dew_temperature','precip_depth_1_hr','wind_direction','wind_speed','month','weekday','hour','is_holiday','season','IsDay','relative_humidity']  
, categorical_feature = ['primary_use','is_holiday','season']
)

In [ ]:
params = {
'task': 'train'
, 'boosting_type': 'gbdt'
, 'objective': 'regression' 
, 'metric': 'rmsle'
, 'subsample': 0.8
,'feature_fraction':0.8
,'bagging_freq': 1
, 'min_data': 1
, 'verbose': -1
}
 
gbm_f = lgb.train(params, lgb_train_f, num_boost_round=500)

/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


In [ ]:
gbm_cv_f=gbm_f.predict(X_cv_f)
# # Normalizing meter_reading column using 0-1 scaling normalization
from sklearn.preprocessing import minmax_scale
Y_cv_f = minmax_scale(Y_cv_f, feature_range=(0, 1))  # 0-1 scaling
gbm_cv_f = minmax_scale(gbm_cv_f, feature_range=(0, 1))  # 0-1 scaling

cv_RMSLE.append(np.sqrt(mean_squared_log_error(Y_cv_f, gbm_cv_f)))
print(cv_RMSLE)

[0.05515768273307979]


In [ ]:
import joblib
filename_reg='lgb_reg_500.sav'
joblib.dump(gbm_f,filename_reg)

['lgb_reg_500.sav']

**Decision Tree Regressor**

In [ ]:
#  As the plot for depth = 15 DTR gives min cross validation error
dtr=DecisionTreeRegressor(max_depth=15,random_state=5)
dtr.fit(X_train_f, Y_train_f)
# dtr_pred_tr=dtr.predict(X_train)
dtr_pred_cv=dtr.predict(X_cv_f)
from sklearn.preprocessing import minmax_scale
dtr_pred_cv = minmax_scale(dtr_pred_cv, feature_range=(0, 1))  # 0-1 scaling
Y_cv_f = minmax_scale(Y_cv_f, feature_range=(0, 1))  # 0-1 scaling
# print("Train rmsle for max_depth: 15",'is',np.sqrt(mean_squared_log_error(Y_train, dtr_pred_tr)))
print("CV rmsle for max_depth: 15",'is',np.sqrt(mean_squared_log_error(Y_cv_f, dtr_pred_cv)))

CV rmsle for max_depth: 15 is 0.06066264852232549


**MLP Regressor**

In [ ]:
!pip install keras
# ! c:\python\python391\python.exe -m pip install --upgrade pip
!pip3 install tenserflow
!pip3 install tqdm

ERROR: Could not find a version that satisfies the requirement tenserflow (from versions: none)
ERROR: No matching distribution found for tenserflow


In [ ]:
from keras import Sequential
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout,BatchNormalization
from keras.initializers import glorot_uniform
from keras.optimizers import adam_v2
from keras.regularizers import l1
from tqdm import tqdm 
import tensorflow as tf

In [ ]:
#Source: https://stackoverflow.com/questions/43855162/rmse-rmsle-loss-function-in-keras
def root_mean_squared_log_error_K(y_true, y_pred):
    msle = tf.keras.losses.MeanSquaredLogarithmicError()
    return K.sqrt(msle(y_true, y_pred)) 

In [ ]:
# Source: https://medium.com/analytics-vidhya/energy-prediction-of-buildings-ashrae-5d873cf038f1
inp_dim=X_train_f.shape[1]
earlystop= EarlyStopping(monitor='val_loss', mode='min', patience=3)

model=Sequential()
model.add(Dense(2048, activation='relu', input_shape=(inp_dim,)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='linear'))
opt=adam_v2.Adam(0.0001)
model.compile(optimizer=opt, loss= root_mean_squared_log_error_K)
model.fit(X_train_f, Y_train_f,epochs=15,batch_size=2048,validation_data=(X_cv_f,Y_cv_f),callbacks=earlystop)

Epoch 1/15
7155/7155 [==============================] - 2246s 314ms/step - loss: 1.1695 - val_loss: 1.2157
Epoch 2/15
7155/7155 [==============================] - 2252s 315ms/step - loss: 1.1757 - val_loss: 0.9787
Epoch 3/15
7155/7155 [==============================] - 2282s 319ms/step - loss: 0.8367 - val_loss: 0.6576
Epoch 4/15
7155/7155 [==============================] - 2242s 313ms/step - loss: 0.6545 - val_loss: 0.6597
Epoch 5/15
7155/7155 [==============================] - 2239s 313ms/step - loss: 0.6528 - val_loss: 0.6581
Epoch 6/15
7155/7155 [==============================] - 2229s 312ms/step - loss: 0.6512 - val_loss: 0.6538
Epoch 7/15
7155/7155 [==============================] - 2243s 313ms/step - loss: 0.6268 - val_loss: 0.4951
Epoch 8/15
7155/7155 [==============================] - 2237s 313ms/step - loss: 0.4910 - val_loss: 0.4979
Epoch 9/15
7155/7155 [==============================] - 2241s 313ms/step - loss: 0.4891 - val_loss: 0.4954
Epoch 10/15
7155/7155 [==============

**CV Results**

In [ ]:
# !pip install prettytable
from prettytable import PrettyTable
summary = PrettyTable()


summary.field_names = ["Model", "RMSLE with min-max scaling"]
summary.add_row(["Baseline 1",0.08995653])
summary.add_row(["Baseline 2",0.079828836])
summary.add_row(["Decision Tree Regressor", 0.057975262262512504])
summary.add_row(["LightGBM GBDT", 0.05515768273307979])
summary.add_row(["Catboost GBDT", 0.04993135674878834])
summary.add_row(["MLP (without min-max scaling)", 0.4810])
print(summary)

+-------------------------------+----------------------------+
|             Model             | RMSLE with min-max scaling |
+-------------------------------+----------------------------+
|           Baseline 1          |         0.08995653         |
|           Baseline 2          |        0.079828836         |
|    Decision Tree Regressor    |    0.057975262262512504    |
|         LightGBM GBDT         |    0.05515768273307979     |
|         Catboost GBDT         |    0.04993135674878834     |
| MLP (without min-max scaling) |           0.481            |
+-------------------------------+----------------------------+


**Predicting results on test data**

In [ ]:
# Reding pre-processed test data
test_merged_F= pd.read_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/test_merged_F.ftr')

In [ ]:
test_merged_F.head()

,index,row_id,meter,site_id,primary_use,square_feet,year_built,floor_count,cloud_coverage,dew_temperature,precip_depth_1_hr,wind_direction,wind_speed,month,weekday,hour,is_holiday,season,IsDay,relative_humidity
0,0,0,0,0,0,8.913685,2008.0,0.693359,4.0,11.703125,0.0,100.0,3.599609,1,6,0,0,3,0,67.5
1,1,1,0,0,0,7.908755,2004.0,0.693359,4.0,11.703125,0.0,100.0,3.599609,1,6,0,0,3,0,67.5
2,2,2,0,0,0,8.589886,1991.0,0.693359,4.0,11.703125,0.0,100.0,3.599609,1,6,0,0,3,0,67.5
3,3,3,0,0,0,10.072639,2002.0,0.693359,4.0,11.703125,0.0,100.0,3.599609,1,6,0,0,3,0,67.5
4,4,4,0,0,0,11.666573,1975.0,0.693359,4.0,11.703125,0.0,100.0,3.599609,1,6,0,0,3,0,67.5


**Prediction on LGB**

In [ ]:
predictions_lgb=pd.DataFrame()
predictions_lgb['row_id']=test_merged_F['row_id']

In [ ]:
test_merged_F.drop(['row_id'], axis=1, inplace=True)

In [ ]:
test_merged_F.drop(['index'], axis=1, inplace=True)

In [ ]:
predictions_lgb['meter_reading_log']= gbm_f.predict(test_merged_F)

In [ ]:
# The prediction are in log form so converting them to exp(x)-1
predictions_lgb['meter_reading']= np.expm1(predictions_lgb['meter_reading_log'])

In [ ]:
predictions_lgb.drop(['meter_reading_log'], axis=1,inplace=True)

In [ ]:
predictions_lgb.head()

,row_id,meter_reading
0,0,8.973302
1,1,29.580018
2,2,59.311757
3,3,25.204743
4,4,25.204743


In [ ]:
predictions_lgb.shape

(41697600, 2)

In [ ]:
predictions_lgb.to_csv('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/LGBM_500.csv', index=False)

**Prediction on DTR**

In [ ]:
predictions_dtr=pd.DataFrame()
predictions_dtr['row_id']=test_merged_F['row_id']

In [ ]:
predictions_dtr['meter_reading_log']= dtr.predict(test_merged_F)

In [ ]:
predictions_dtr['meter_reading']= np.expm1(predictions_dtr['meter_reading_log'])

In [ ]:
predictions_dtr.drop(['meter_reading_log'], axis=1,inplace=True)

In [ ]:
predictions_dtr.head()

,row_id,meter_reading
0,0,59.382707
1,1,12.286099
2,2,4.371649
3,3,426.977239
4,4,77.811900


In [ ]:
predictions_dtr.to_csv('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/DTR_15.csv', index=False)

**Prediction on Catboost**

In [ ]:
predictions_cat=pd.DataFrame()
predictions_cat['row_id']=test_merged_F['row_id']

In [ ]:
predictions_cat['meter_reading_log']= cgbdt_f.predict(test_merged_F)

In [ ]:
predictions_cat['meter_reading']= np.expm1(predictions_cat['meter_reading_log'])

In [ ]:
predictions_cat.drop(['meter_reading_log'], axis=1,inplace=True)

In [ ]:
pd.DataFrame(predictions_cat['meter_reading']<0).head()

,meter_reading
0,False
1,False
2,False
3,False
4,False


In [ ]:
predictions_cat['meter_reading'] = predictions_cat['meter_reading'].clip(lower=0)

In [ ]:
predictions_cat.shape

(41697600, 2)

In [ ]:
predictions_cat.to_csv('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/Catboost_1200_clipped.csv', index=False)

**Adding cyclical time features on hour column (No improvement in results)**

In [ ]:
# Adding cyclical time features on hour column
# X_train_f, Y_train_f, X_cv_f
X_train_f["hour_sin"] = X_train_f["hour"].apply(lambda x:np.sin(2*np.pi*x/24))
X_train_f["hour_cos"] = X_train_f["hour"].apply(lambda x:np.cos(2*np.pi*x/24))
X_cv_f["hour_sin"] = X_cv_f["hour"].apply(lambda x:np.sin(2*np.pi*x/24))
X_cv_f["hour_cos"] = X_cv_f["hour"].apply(lambda x:np.cos(2*np.pi*x/24))

test_merged_F["hour_sin"] = test_merged_F["hour"].apply(lambda x:np.sin(2*np.pi*x/24))
test_merged_F["hour_cos"] = test_merged_F["hour"].apply(lambda x:np.cos(2*np.pi*x/24))


In [ ]:
X_train_f.head()

,meter,site_id,primary_use,square_feet,year_built,floor_count,cloud_coverage,dew_temperature,precip_depth_1_hr,wind_direction,wind_speed,month,weekday,hour,is_holiday,season,IsDay,relative_humidity,hour_sin,hour_cos
0,0,0,11,9.110078,2016.0,0.693359,6.0,20.000000,0.0,0.0,0.000000,1,4,0,0,3,0,73.8125,0.0,1.0
1,0,13,6,12.427046,1965.0,0.693359,8.0,-12.203125,0.0,250.0,4.101562,1,4,0,0,3,0,73.3750,0.0,1.0
2,2,13,8,11.337619,1965.0,0.693359,8.0,-12.203125,0.0,250.0,4.101562,1,4,0,0,3,0,73.3750,0.0,1.0
3,1,13,8,11.337619,1965.0,0.693359,8.0,-12.203125,0.0,250.0,4.101562,1,4,0,0,3,0,73.3750,0.0,1.0
4,0,13,8,11.337619,1965.0,0.693359,8.0,-12.203125,0.0,250.0,4.101562,1,4,0,0,3,0,73.3750,0.0,1.0


In [ ]:
# X_train_f.reset_index().to_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/X_train_F1.ftr')
# X_cv_f.reset_index().to_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/X_cv_F1.ftr')
# test_merged_F.reset_index().to_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/test_merged_F1.ftr')

In [ ]:
test_merged_F= pd.read_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/test_merged_F1.ftr')
X_train_f= pd.read_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/X_train_F1.ftr')
X_cv_f= pd.read_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/X_cv_F1.ftr')

In [ ]:
test_merged_F= pd.read_feather('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/test_merged_F1.ftr')
test_merged_F.drop(['index'], axis=1, inplace=True)

In [ ]:
test_merged_F.drop(['index'], axis=1, inplace=True)
X_train_f.drop(['index'], axis=1, inplace=True)
X_cv_f.drop(['index'], axis=1, inplace=True)

In [ ]:
test_merged_F.head()

,row_id,meter,site_id,primary_use,square_feet,year_built,floor_count,cloud_coverage,dew_temperature,precip_depth_1_hr,wind_direction,wind_speed,month,weekday,hour,is_holiday,season,IsDay,relative_humidity,hour_sin,hour_cos
0,0,0,0,0,8.913685,2008.0,0.693359,4.0,11.703125,0.0,100.0,3.599609,1,6,0,0,3,0,67.5,0.0,1.0
1,1,0,0,0,7.908755,2004.0,0.693359,4.0,11.703125,0.0,100.0,3.599609,1,6,0,0,3,0,67.5,0.0,1.0
2,2,0,0,0,8.589886,1991.0,0.693359,4.0,11.703125,0.0,100.0,3.599609,1,6,0,0,3,0,67.5,0.0,1.0
3,3,0,0,0,10.072639,2002.0,0.693359,4.0,11.703125,0.0,100.0,3.599609,1,6,0,0,3,0,67.5,0.0,1.0
4,4,0,0,0,11.666573,1975.0,0.693359,4.0,11.703125,0.0,100.0,3.599609,1,6,0,0,3,0,67.5,0.0,1.0


In [ ]:
cv_RMSLE=[]
lgb_train_f = lgb.Dataset(X_train_f, Y_train_f
,feature_name =  ['meter','site_id','primary_use','square_feet','year_built','floor_count','cloud_coverage','dew_temperature','precip_depth_1_hr','wind_direction','wind_speed','month','weekday','hour','is_holiday','season','IsDay','relative_humidity','hour_sin','hour_cos']  
, categorical_feature = ['meter','site_id','primary_use','year_built','month','weekday','hour','is_holiday','season','IsDay']
)

In [ ]:
features=  ['meter','site_id','primary_use','square_feet','year_built','floor_count','cloud_coverage','dew_temperature','precip_depth_1_hr','wind_direction','wind_speed','month','weekday','hour','is_holiday','season','IsDay','relative_humidity','hour_sin','hour_cos']  
categorical_feature = ['meter','site_id','primary_use','year_built','month','weekday','hour','is_holiday','season','IsDay']

In [ ]:
params = {
'task': 'train'
, 'boosting_type': 'gbdt'
, 'objective': 'regression' 
, 'metric': 'rmse'
,'learning_rate' : 0.01
,'num_iterations' : 1024
,'num_leaves' : 64

}
 
gbm_f = lgb.train(params, lgb_train_f, num_boost_round=500)

In [ ]:
import joblib
gbm_f= joblib.load('/content/drive/MyDrive/PGDUOH/Project/lgbm_periodic.sav')

In [ ]:
gbm_cv_f=gbm_f.predict(X_cv_f)
# # Normalizing meter_reading column using 0-1 scaling normalization
from sklearn.preprocessing import minmax_scale
Y_cv_f = minmax_scale(Y_cv_f, feature_range=(0, 1))  # 0-1 scaling
gbm_cv_f = minmax_scale(gbm_cv_f, feature_range=(0, 1))  # 0-1 scaling

cv_RMSLE.append(np.sqrt(mean_squared_log_error(Y_cv_f, gbm_cv_f)))
print(cv_RMSLE)

[0.06553721967841396]


In [ ]:
# import joblib
# filename_reg='/content/drive/MyDrive/PGDUOH/Project/lgbm_periodic.sav'
# joblib.dump(gbm_f,filename_reg)

In [ ]:
predictions_lgbm=pd.DataFrame()
predictions_lgbm['row_id']=test_merged_F['row_id']

In [ ]:
test_merged_F.drop(['row_id'], axis=1, inplace=True)

In [ ]:
test_merged_F.head()

,meter,site_id,primary_use,square_feet,year_built,floor_count,cloud_coverage,dew_temperature,precip_depth_1_hr,wind_direction,wind_speed,month,weekday,hour,is_holiday,season,IsDay,relative_humidity,hour_sin,hour_cos
0,0,0,0,8.913685,2008.0,0.693359,4.0,11.703125,0.0,100.0,3.599609,1,6,0,0,3,0,67.5,0.0,1.0
1,0,0,0,7.908755,2004.0,0.693359,4.0,11.703125,0.0,100.0,3.599609,1,6,0,0,3,0,67.5,0.0,1.0
2,0,0,0,8.589886,1991.0,0.693359,4.0,11.703125,0.0,100.0,3.599609,1,6,0,0,3,0,67.5,0.0,1.0
3,0,0,0,10.072639,2002.0,0.693359,4.0,11.703125,0.0,100.0,3.599609,1,6,0,0,3,0,67.5,0.0,1.0
4,0,0,0,11.666573,1975.0,0.693359,4.0,11.703125,0.0,100.0,3.599609,1,6,0,0,3,0,67.5,0.0,1.0


In [ ]:
predictions_lgbm['meter_reading_log']= gbm_f.predict(test_merged_F)

In [ ]:
predictions_lgbm['meter_reading']= np.expm1(predictions_lgbm['meter_reading_log'])

In [ ]:
predictions_lgbm.drop(['meter_reading_log'], axis=1,inplace=True)

In [ ]:
predictions_lgbm['meter_reading'] = predictions_lgbm['meter_reading'].clip(lower=0)

In [ ]:
predictions_lgbm.shape

(41697600, 2)

In [ ]:
predictions_lgbm.to_csv('/content/drive/MyDrive/PGDUOH/Project/ProcessedData/lgbm_periodic.csv', index=False)